In [2]:
import numpy as np
from numpy import genfromtxt
from sklearn.preprocessing import MinMaxScaler

# Question 1

In [2]:
A = genfromtxt('life_expectancy_X.csv', delimiter=',')
A_test = genfromtxt('life_expectancy_X_test.csv', delimiter=',')

y = genfromtxt('life_expectancy_y.csv', delimiter=',')
y_test = genfromtxt('life_expectancy_y_test.csv', delimiter=',')

# Scale
scaler = MinMaxScaler()
A = scaler.fit_transform(A)
A_test = scaler.transform(A_test)

### 1)

In [3]:
def f_prime(w, phi, y, l1_lambda):
    return (1/len(phi))*(phi.T @ (phi@w - y)) + l1_lambda * np.sign(w)

def gradient_descent(phi, y, w, eta, num_iterations, l1_lambda):
	for i in range(num_iterations):
		w = w - eta * f_prime(w, phi, y, l1_lambda)  
            
	return w

def mse_func(phi, w, y):
	return np.mean((phi@w - y)**2)

def sorted_features_importance(feature_names_list, w, lin_reg=False):
	if lin_reg == True:
		sorted_w = w[:-1]
	else:
		sorted_w = w

	sorted_indices = np.argsort(-np.abs(sorted_w))
	sorted_features = [feature_names_list[i] for i in sorted_indices]
	sorted_w = w[sorted_indices]

	for feature, weight in zip(sorted_features, sorted_w):
		print(f"{feature}: {weight:.4f}")

In [4]:
phi = np.hstack((np.ones((A.shape[0], 1)), A))
w0 = np.zeros(phi.shape[1])
eta = 0.02
l1_lambda = 0.01

w = gradient_descent(phi, y, w0, eta, 10000, l1_lambda)

phi_test = np.hstack((np.ones((A_test.shape[0], 1)), A_test))

print(f'MSE: {mse_func(phi_test, w, y_test)}')

MSE: 0.005947909788639809


### 2)

In [5]:
feature_names = [
    "Exercise amount",
    "Amount of supportive relationships",
    "Number of siblings",
    "Alcohol / Drugs / Smoking consumption",
    "Height",
    "Attractiveness",
    "work ethics"
]

sorted_features_importance(feature_names, w, lin_reg=True)

Exercise amount: 59.8926
Amount of supportive relationships: 29.9769
Number of siblings: 14.9417
Height: -9.8219
Attractiveness: 0.0002
Alcohol / Drugs / Smoking consumption: 0.0001
work ethics: -0.0000


Biggest Positive Impact: Exercise Amount, Supportive Relationshops, Number of Siblings  
Negative Impact: Height  
Attractiveness, Alcohol/Drugs/Smoking, and Work Ethic all have a minimal impact on longetivty  

### 3)

In [6]:
def center_func(n):
      C = np.eye(n) - (1/n) * np.ones((n, n))
      return C

def pca_reduc_func(X, num_dimensions):
    n = len(X)

    C = center_func(n)
    Q = X.T @ C @ X

    [D, V] = np.linalg.eigh(Q)

    v = V[:, -num_dimensions:]

    X_hat = X @ v 

    return X_hat, v, D 

def f_prime(w, phi, y):
    return (1/len(phi))*(phi.T @ (phi@w - y))

def gradient_descent(phi, y, w, eta, num_iterations):
    for i in range(num_iterations):
        w = w - eta * f_prime(w, phi, y)  

    return w

In [14]:
pca_A, pca_v, eig_vals = pca_reduc_func(A, 4)

pca_phi = np.hstack((np.ones((pca_A.shape[0], 1)), pca_A))
w0 = np.zeros(pca_phi.shape[1])
eta = 0.05

pca_w = gradient_descent(pca_phi, y, w0, eta, 20000)

A_test_C = center_func(len(A_test))
pca_A_test = A_test @ pca_v

pca_phi_test = np.hstack((np.ones((pca_A_test.shape[0], 1)), pca_A_test))
print(f'Test MSE: {mse_func(pca_phi_test, pca_w, y_test)}')
# print(f'MSE: {mse_func(pca_phi, pca_w, y)}')

Test MSE: 58.21643737770854


In [31]:
feature_contributions = np.mean(pca_v, axis=1)
feature_contributions

array([ 0.09306744,  0.2454878 , -0.25985832,  0.05647923, -0.18328759,
       -0.27576156, -0.02681717])

### 4)
PCA probably is not a good idea for this problem as shown by the much higher MSE than regular LASSO Linear Regression. PCA tries to maximize variance but does not account for the labels. Therefore, features which have low variance but high importance for predicting longetivity will have their importance reduce when PCA is applied. For example, number of sibilings and also relationships probably have a low variance but are of high importance to predicting longetivity, so with PCA their importance would be reduced. Therefore, using linear regression is not a good predictor for this problem and should not be used.

In [34]:
def cov_func(X, Y):
    n = len(X)
    x_mean = np.mean(X)
    y_mean = np.mean(Y)

    return (1/n) * sum((X[i] - x_mean)*(Y[i] - y_mean) for i in range(n))


def corr_func(X, Y):
    return (cov_func(X, Y))/(np.sqrt((cov_func(X, X))*(cov_func(Y, Y))))

corr_list = []
for xi in A.T:
    corr = corr_func(xi, y)
    corr_list.append(corr)

print(corr_list)

[0.8508629239616721, 0.43643566943179135, 0.002459752414860447, -0.3122734519911499, -0.036254143816136813, -0.03482415007593405, 0.03136403248523949]


In [35]:
# Check correlations with feature names
feature_names = [
    "Exercise amount",
    "Amount of supportive relationships",
    "Number of siblings",
    "Alcohol / Drugs / Smoking consumption",
    "Height",
    "Attractiveness",
    "work ethics"
]

sorted_features_importance(feature_names, np.array(corr_list))

Exercise amount: 0.8509
Amount of supportive relationships: 0.4364
Alcohol / Drugs / Smoking consumption: -0.3123
Height: -0.0363
Attractiveness: -0.0348
work ethics: 0.0314
Number of siblings: 0.0025


Exercise amount and supportive relationships have a significant positive correlation with longetivty.  
Alcohol/drugs has a fairly siginificant negative correlation with longetivty.  
Height, attractivness, work ethics, and number of siblings have a minimal correlation with longeivity.

### 6)

Both excericise amount and amount of supportive relationshops have a big postitive impact according to correlation and the LASSO weights. However, Alcohol/drugs consumption has a decently high negative correlation, while the LASSO weight is basically zero. Furthermore, according to the weights, number of siblings is the third most important feature, but it has a very low correlation.

In [ ]:
# Use eigenvalues to find most impactful features for PCA
eig_features = np.mean(np.abs(pca_v), axis=1)

sorted_features_importance(feature_names, eig_features)

Alcohol / Drugs / Smoking consumption: 0.3101
Attractiveness: 0.3002
Number of siblings: 0.2965
Amount of supportive relationships: 0.2455
work ethics: 0.2313
Height: 0.2184
Exercise amount: 0.1348


For PCA, the order of impactfulness is: alcohol consumption, attractiveness, number of siblings, amount of relationships, work ethic, height, and finally exercise amount. This is the opposite of correlation and LASSO weights, as exercise is the most impactful feature according to those metrics. Attractiveness is also low for both of those metrics, while attractiveness has the second biggest impact on PCA. Alcohol consumption has a somewhat high negative correlation, and is also the most impactful PCA feature. Overall, the correlation table matches the LASSO weights more than the PCA feature importance.

# Question 2

In [3]:
A = genfromtxt('gpa_prediction_X.csv', delimiter=',')
A_test = genfromtxt('gpa_prediction_X_test.csv', delimiter=',')

y = genfromtxt('gpa_prediction_y.csv', delimiter=',')
y_test = genfromtxt('gpa_prediction_y_test.csv', delimiter=',')

# Scale
scaler = MinMaxScaler()
A = scaler.fit_transform(A)
A_test = scaler.transform(A_test)

### 1)

In [17]:
def accuracy_func(predictions, y):
    return np.mean(predictions == y)

# Logistic Regression
def f_prime(w, phi, y, l1_lambda):
    return 1/len(phi) * phi.T @ (sigmoid(phi, w) - y) + (l1_lambda * np.sign(w))

def sigmoid(phi, w):
    return 1/(1 + np.exp(-phi @ w))

def gradient_descent(phi, y, w, eta, num_iterations, l1_lambda=0):
	for i in range(num_iterations):
		w = w - eta * f_prime(w, phi, y, l1_lambda)  
            
	return w

def log_reg_pred(phi, w):
      return np.where(sigmoid(phi, w) >= 0.5, 1, 0)


# def run_lambda_values(w0, X, y, lambdas, eta):
#     best_acc = -np.inf  # store initial accuracy as high value
#     best_lambda = None
#     best_w = w0


#     for l1_lambda in lambdas:
        # w = gradient_descent(A, y, w0, eta, 10000, l1_lambda)

        # predictions = log_reg_pred(A, w)
        # acc = accuracy_func(predictions, y)

    #     if acc > best_acc:  # if new accuracy is better, store value
    #         best_acc = acc
    #         best_lambda = l1_lambda
    #         best_w = w

    # return best_w, best_lambda, best_acc

In [18]:
phi = np.hstack((np.ones((A.shape[0], 1)), A))

w0 = np.zeros(phi.shape[1])
eta = 0.002
l1_lambda = 0.02

w = gradient_descent(phi, y, w0, eta, 10000, l1_lambda)

# Accuracy using test
phi_test = np.hstack((np.ones((A_test.shape[0], 1)), A_test))
predictions = log_reg_pred(phi_test, w)
acc = accuracy_func(predictions, y_test)
print(f'Test Accuracy: {acc}')

Test Accuracy: 0.955


### 2)

In [19]:
def sorted_features_importance(feature_names_list, w, lin_reg=False):
	if lin_reg == True:
		sorted_w = w[:-1]
	else:
		sorted_w = w

	sorted_indices = np.argsort(-np.abs(sorted_w))
	sorted_features = [feature_names_list[i] for i in sorted_indices]
	sorted_w = w[sorted_indices]

	for feature, weight in zip(sorted_features, sorted_w):
		print(f"{feature}: {weight:.4f}")

In [21]:
features = ["Exercise amount",
"Amount of supportive relationships",
"Number of siblings",
"Alcohol / Drugs / Smoking consumption",
"Height",
"Attractiveness",
"work ethics"]

sorted_features_importance(features, w, lin_reg=True)

Height: -1.1829
Number of siblings: 0.1943
Exercise amount: 0.0001
Attractiveness: 0.0000
work ethics: -0.0000
Alcohol / Drugs / Smoking consumption: -0.0000
Amount of supportive relationships: -0.0000


The number of siblings postiviely impacts your grade, while height has a huge negative impact. Supportive relationships, attractiveness, alcohol consumption, exercise amount, and work ethic all have minimal impact on your GPA according to the logistic regression weights.

### 3)

In [16]:
def center_func(n):
      C = np.eye(n) - (1/n) * np.ones((n, n))
      return C

def lda_q_func(X, y):
      X0 = X[y == 0]  
      X1 = X[y == 1]

      c0 = np.reshape(np.mean(X0, axis=0), (X0.shape[1], 1))
      c1 = np.reshape(np.mean(X1, axis=0), (X1.shape[1], 1))

      u = c0 - c1
      S1 = u @ (u.T)

      C0 = center_func(len(X0))
      C1 = center_func(len(X1))

      S2 = (X0.T @ C0 @ X0) + (X1.T @ C1 @ X1)

      Q = np.linalg.inv(S2) @ S1
      return Q

def lda_eig(X, y, num_dimensions):
      Q = lda_q_func(X, y)

      [D, V] = np.linalg.eigh(Q)

      v = V[:, -num_dimensions:]

      X_hat = X @ v 

      return X_hat, v, D 

In [22]:
lda_A, lda_v, lda_eig_vals = lda_eig(A, y, 3)

lda_phi = np.hstack((np.ones((lda_A.shape[0], 1)), lda_A))
w0 = np.zeros(lda_phi.shape[1])
eta = 0.05

lda_w = gradient_descent(lda_phi, y, w0, eta, 20000)

# Test accuracy
lda_A_test = A_test @ lda_v
lda_phi_test = np.hstack((np.ones((lda_A_test.shape[0], 1)), lda_A_test))

lda_predictions = log_reg_pred(lda_phi_test, lda_w)
acc = accuracy_func(lda_predictions, y_test)
print(f'Test Accuracy using LDA: {acc}')

Test Accuracy using LDA: 0.975


### 4)

In [31]:
lda_w_proj = lda_v @ lda_w[:-1]

sorted_features_importance(features, lda_w_proj)

Alcohol / Drugs / Smoking consumption: -0.7128
Amount of supportive relationships: 0.3844
Attractiveness: -0.1825
work ethics: 0.1543
Height: 0.1333
Number of siblings: 0.1111
Exercise amount: 0.0183


### 5)

The important factors in LDA are much different than the important ones for logistic regression. The only factors with any signficant weight according to the original logistic regression were Height, which had a negative impact and number of sibilings, which had a small positive impact. However, after LDA, Alcohol consumption had a big negative weight, then number of supportive relationships had a positive weight, both of which had no importance at all before LDA. Number of siblins and height both had a small weight for LDA, but not negligible. Exercise amount had a low impact for both logistic regression, and LDA.